# Human DNA

import libraries for later use

In [ ]:
import numpy as np
import pandas as pd
import sklearn

In [ ]:
df_train = pd.read_csv('exercise_data/human_dna_train_split.csv')
df_valid = pd.read_csv('exercise_data/human_dna_validation_split.csv')
df_test  = pd.read_csv('exercise_data/human_dna_test_split.csv')

In [ ]:
print(f"Size of training_set: {df_train.shape}")
print(f"Shape of test set: {df_test.shape}")
print(f"Shape of validation set: {df_valid.shape}")

### UnderSampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler, CondensedNearestNeighbour

In [ ]:
def undersample_dataframe(df):
    X = np.vstack(df['sequences'].values)
    y = df['labels'].values
    undersampler = RandomUnderSampler(sampling_strategy=0.05)
    X_undersampled, y_undersampled = undersampler.fit_resample(X, y)
    undersampled_df = pd.DataFrame({'labels':y_undersampled, 'sequences': X_undersampled[:,0]})
    return undersampled_df

In [ ]:
print(f"Size before undersampling {df_train.shape[0]}")
df_train_undersampled = undersample_dataframe(df_train)
print(f"Size after undersampling {df_train_undersampled.shape[0]}")

### Mapping DNA to a vector

We will map the DNA into a vector, by mapping each Character (A,T,C,G) into a one-hot vector and then concatonating all these vectors together. As we have a string of 398 Characters this gives us a final vector of length 1592. 

*This operation takes some time, please be patient*

In [ ]:
import utility
df_train_undersampled['sequences'] = df_train_undersampled['sequences'].map(utility.map_dna_into_vector)
df_valid['sequences'] = df_valid['sequences'].map(utility.map_dna_into_vector)
df_test['sequences']  = df_test['sequences'].map(utility.map_dna_into_vector)

## PCA

At first it seemed like to be a good idea to compress the Vector of length 1592 to the length of 200. Which seemed to be reasonable given to plot PCA produces. It did help somewhat with the training-time, but the f1-score got way worse <0.08. So we dropped the idea of using PCA for compressing the trainings-data

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pca = PCA()
pca = pca.fit(np.vstack(df_train_undersampled['sequences'].values))

plt.plot(range(len(pca.explained_variance_)), pca.explained_variance_ratio_, )
plt.ylabel('Explained variance ratio')
plt.xlabel('Index of Principal Component')
plt.show()

### Creating a DataFrame for later Evalution

We create the dataframe eval_df for the final evalution. 

In [ ]:
eval_df = pd.DataFrame(data=[], columns=["Name", "AUROC", "AUPRC", "f1_cv", "f1_test"])

## Models

In [ ]:
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import f1_score
from multiprocessing import Pool, cpu_count

We thought that the evaluation of the models was kind of slow so we created a parallel job. Each process evaluates a model using the evaluator function

In [ ]:
def evaluator(args):
    # read out parameters
    model, params, train, valid = args
    
    # get train and validation data
    train_data = np.vstack(train['sequences'].values)
    valid_data = np.vstack(valid['sequences'].values)
    
    # Fit Model with parameters 
    m = model(**params)
    m = m.fit(train_data, train['labels'].values)
    
    # Get prediction of our data
    pred_val = m.predict(valid_data)
    score = f1_score(valid['labels'].values, pred_val)
    return score

In here we evaluate all models with all the different parameters we specified. First we evaluate the model using the training data and the evaluation data. Then we train the best of those models on the combined training and validation data, to get an estimate of the f1-score on the test-data. 

*It's important to note that we evaluate all the models based on their performance on the validation-set.*

In [ ]:
def evaluate_model(model, params, train, valid, test, eval_df):
    # Put Data into a usable Matrix Format
    train_data = np.vstack(train['sequences'].values)
    valid_data = np.vstack(valid['sequences'].values)
    test_data = np.vstack(valid['sequences'].values)
    
    combined_data = np.vstack([train_data, valid_data])
    combined_labels = np.hstack([train['labels'].values, valid['labels'].values])
    
    # Create Instance of the Model
    m = model()
    
    
    # Search for the best params in our model and print the best score
    p = Pool(cpu_count())
    grid = ParameterGrid(params)
    scores = p.map(evaluator, [(model, params, train, valid) for params in grid])
    p.close()
    
    # Get best parameters and scores
    best_score = np.max(scores)
    best_params = grid[np.argmax(scores)]
    print(f"The best score was {best_score}")
    
    
    # Train our best model on the whole train-dataset
    best_estimator = model(**best_params)
    best_estimotor = best_estimator.fit(combined_data, combined_labels)
    
    # Evaluate on the Test set
    pred_val = best_estimator.predict(test_data)
    true_val = test['labels'].values
    auroc, auprc, f1 = utility.get_scores(true_val, pred_val)
    
    # Append to our Dataframe
    eval_df = eval_df.append({'Name': model.__name__, 'AUROC':auroc , 'AUPRC': auprc, 'f1_cv':best_score, "f1_test": f1}, ignore_index=True)
    return (best_estimator, eval_df)

### Logistic Regression

We test logistic Regression with a few different parameters

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params = {
    'penalty': ['l1', 'l2'],
    'C': [1, 10, 100],
    'class_weight': ['balanced'],
    'solver': ['liblinear']
}

In [ ]:
lg_best_estimator, eval_df = evaluate_model(LogisticRegression, params, df_train_undersampled, df_valid, df_test, eval_df)

### SVC

We test Support-Vector-Classifer with different parameters. Be careful, this task takes a long time and uses a lot of memory

In [ ]:
from sklearn.svm import SVC

In [ ]:
params = {'kernel': ['linear', 'rbf', 'poly'],
          'C': [1, 10],
          'class_weight': ['balanced'],
          'gamma': ['auto', 'scale'],
         }

In [ ]:
svc_best_estimator, eval_df = evaluate_model(SVC, params, df_train_undersampled, df_valid, df_test, eval_df)

### Random Forest

We test the RAndom-Forest C

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
params = {
    'n_estimators':[10, 100, 300],    
    'class_weight': ['balanced', 'balanced_subsample']
}

In [ ]:
rfc_best_estimator, eval_df = evaluate_model(RandomForestClassifier, params, df_train_undersampled, df_valid, df_test, eval_df)

###  Gaussian Process Classifer

We skip this classifier, as the memory-requirements crashes the kernel with ~64gb ram

### DL Model

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.layers import BatchNormalization,Conv1D,Input,Add,Dense,Flatten
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import Adam


def f1(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = tf.cast(tf.equal(y_true,y_pred),"int32")
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 2 * precision * recall / (1 * precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

def add_RB(x):
    xout=BatchNormalization()(x)
    xout=Conv1D(filters=32,kernel_size=11,dilation_rate=1,padding='same',activation='relu')(x)
    xout=BatchNormalization()(xout)
    xout=Conv1D(filters=32,kernel_size=11,dilation_rate=1,padding='same',activation='relu')(xout)
    return xout

In [ ]:
tf.reset_default_graph()
x=Input(shape=[1592,1])

x1=Conv1D(filters=32,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(x)

xrb=add_RB(x1)
#xrb=add_RB(xrb)
#xrb=add_RB(xrb)
#xrb=add_RB(xrb)

x2=Conv1D(filters=32,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(xrb)
x3=Conv1D(filters=32,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(x1)

xout=Conv1D(filters=1,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(Add()([x2,x3]))
xout=Flatten()(xout)
xout=Dense(units=1,activation='sigmoid')(xout)

model=Model(x,xout)
model.compile(optimizer=Adam(),loss='binary_crossentropy',metrics=[f1])
class_wt={0:1,1:1}

In [ ]:
train_data = np.vstack(df_train['sequences'].values)[:,:,None]
test_data  = np.vstack(df_test['sequences'].values)[:,:,None]

train_val=df_train['labels'].values
train_val[train_val==-1]=0
model.fit(x=train_data,y=train_val,batch_size=64,epochs=20,class_weight=class_wt)

In [ ]:
pred_val=(model.predict(test_data)>0.5).astype(np.int)
true_val=df_test['labels']
true_val[true_val==-1]=0

dl_mtr=utility.get_scores(true_val,pred_val)
dl_f1=utility.f1_score(true_val,pred_val)
eval_df=eval_df.append({'Name':'DL_model','AUROC':dl_mtr[0],'AUPRC':dl_mtr[1],'f1_score':dl_f1},ignore_index=True)

## Evaluation

In [ ]:
eval_df

In [ ]:
best_estimator = svc_best_estimator
predicted_values = best_estimator.predict(np.vstack(test['DNA'].values))
true_values = test['labels']

import matplotlib.pyplot as plt

# compute ROC curve
fpr, tpr, thresholds_roc = roc_curve(true_values, predicted_values)
roc_auc = auc(fpr, tpr)
precision, recall, thresholds_prc = precision_recall_curve(true_values, predicted_values)

# compute precision-recall curve
auprc = auc(recall, precision)
precision_random, recall_random, thresholds_random = precision_recall_curve(true_values, np.random.rand(len(true_values)))
auprc_random = auc(recall_random, precision_random)
other_scores_validation = [roc_auc, auprc, auprc_random]

# plot curves
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
                 lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - Specificity / False Positive Rate')
plt.ylabel('Sensibility / True Positive Rate')
plt.title('Receiver Operating Characteristic curve')
plt.legend(loc="lower right")
#plt.savefig('./models/' + experiment_id + '/' + model_name + '_roc_curve_validation.png')

plt.figure()
lw = 2
plt.plot(recall, precision, color='darkorange',
                     lw=lw, label='AUPRC curve (area = %0.2f)' % auprc)
plt.plot(recall_random, precision_random, color='navy', linestyle='--',
                     lw=lw, label='random AUPRC curve (area = %0.2f)' % auprc_random)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend(loc="lower right")

In [ ]:
def save_hidden_test_results(result_1d_array):

    hidden_split_length = 30021

    if len(np.shape(result_1d_array)) > 1:
        raise Exception("The array must be one dimensional.")

    if len(result_1d_array) != hidden_split_length:
        raise Exception("The array doesn't have the proper length.")
        
    np.save('./hidden_test_result', result_1d_array)

In [ ]:
def save_group_members_and_contributions(members_and_contributions):
    
    if len(members_and_contributions) > 4:
        raise Exception("The maximum number of allowed members is 4.")
        
    if len(members_and_contributions) < 2:
        raise Exception("The minimum number of allowed members is 2.")
        
    df = pd.DataFrame(members, columns=['name','contribution'])
    df.to_csv('./members_and_contributions.csv', index=False)

    return

In [ ]:
hidden_df = pd.read('./exercise_data/human_dna_test_hidden.csv')
hidden_values = np.vstack(hidden_df['sequences'].values)
predicted_values = best_estimator.predict(hidden_values)
save_hidden_test_results(predicted_values)

In [ ]:
members = [['Karthik Pattisapu', 'Deep Learning Model'], 
           ['Juan Lopez', 'String Kernels Shogun'],
           ['Martin Villavicencio', 'Sklearn Methods']]
save_group_members_and_contributions(members)